In [83]:
import pandas as pd
import numpy as np
from rsome import ro
from rsome import grb_solver as grb

In [84]:
# Open file
df = pd.read_csv("data.csv")

# Data Preparation
* In this section, further cleaning of data was performed.

In [85]:
# Fill NA values with 0
df = df.fillna(0)

# Drop columns that will not be used
df = df.drop(df.columns[[0, 3, 4, 5, 6, 7, 8, 9, 10]], axis=1)

# Rename columns to include the units
df.rename(columns={
    'product name ': 'product_name',
    'quantity (CLEANED to Gram)':'quantity-g_value',
    'fat_value': 'fat_value-g_value',
    'saturated-fat_value': 'saturated-fat-g_value',
    'carbohydrates_value': 'carbohydrates-g_value', 
    'sugars_value': 'sugars_g_value',
    'proteins_value': 'proteins-g_value',
    'salt value': 'salt-g_value', 
    'sodium_value': 'sodium-g_value'
    }, inplace=True
)
# Drop units column
df = df.drop(columns=['carbohydrates_unit', 'sugars_unit',
             'proteins_unit', 'salt_unit', 'sodium_unit', 'fat_unit', 'energy-kcal_unit', 'saturated-fat_unit'])

# Get pdt_Id and shift to first columns for easier viewing
df['pdt_Id'] = np.arange(df.shape[0])
first_column = df.pop('pdt_Id')
df.insert(0, 'pdt_Id', first_column)

df.head()


,pdt_Id,product_name,quantity-g_value,energy-kcal_value,fat_value-g_value,saturated-fat-g_value,carbohydrates-g_value,sugars_g_value,proteins-g_value,salt-g_value,sodium-g_value
0,0,Straight Red Tea,250.0,23.2,0.0,0.0,5.8,5.8,0.0,0.02,0.008
1,1,Linsen Eintopf mit Würstchen,800.0,8.0,2.0,0.8,9.4,1.3,4.2,0.98,0.392
2,2,Fusilli,500.0,359.0,2.0,0.5,71.0,3.5,13.0,0.01,0.004
3,3,Essiggurken,670.0,25.0,0.5,0.1,4.3,4.2,0.8,1.30,0.520
4,4,Jelly belly,70.0,356.0,0.0,0.0,89.0,60.0,0.2,0.80,0.320


# Optimization Modelling

In [86]:
df

,pdt_Id,product_name,quantity-g_value,energy-kcal_value,fat_value-g_value,saturated-fat-g_value,carbohydrates-g_value,sugars_g_value,proteins-g_value,salt-g_value,sodium-g_value
0,0,Straight Red Tea,250.0,23.2,0.0,0.0,5.8,5.8,0.0,0.0200,0.008
1,1,Linsen Eintopf mit Würstchen,800.0,8.0,2.0,0.8,9.4,1.3,4.2,0.9800,0.392
2,2,Fusilli,500.0,359.0,2.0,0.5,71.0,3.5,13.0,0.0100,0.004
3,3,Essiggurken,670.0,25.0,0.5,0.1,4.3,4.2,0.8,1.3000,0.520
4,4,Jelly belly,70.0,356.0,0.0,0.0,89.0,60.0,0.2,0.8000,0.320
...,...,...,...,...,...,...,...,...,...,...,...
421,421,Toast cocoa,160.0,317.0,0.0,2.0,8.0,1.0,45.0,1.3500,0.540
422,422,Orhanic beetroot hummus,200.0,114.5,0.0,1.0,7.0,7.0,3.5,1.0750,0.430
423,423,Rice cracker,90.0,124.0,0.0,1.9,79.0,0.5,12.3,1.2325,0.493
424,424,Chickened out Chunks,160.0,1.0,0.0,0.4,2.9,0.5,21.3,1.1100,0.444


In [87]:
# Declare Final Variables
#Calories per household for 30 days (calories)
total_cal = 30*2*(1800+2200)
min_cal = 30*4*(1200)

#Fat per household for 30 days (grams)
total_fat = 30*2*(60+74) 

#Saturated fat per household for 30 days (grams)
total_satfat = 30*2*(24+20) 

#Carbohydrates per household for 30 days(grams)
total_carbs = 30*2*(303+250)

#Sugars per household for 30 days (grams)
total_sugars = 30*4*(45)

#Proteins per household for 30 days (grams)
total_proteins = 30*2*(68+58)

#Fiber per household for 30 days (grams)
total_fiber = 30*2*(25+38)

#Salt per household for 30 days (grams)
total_salt = 30*4*(5)

#Sodium per household for 30 days (grams)
total_sodium = 30*4*(2)

#Max weight that van can take
total_weight = 500

In [88]:
weight_food = df['quantity-g_value']
calories = df['energy-kcal_value'].values
fat = df['fat_value-g_value'].values
sat_fat = df['saturated-fat-g_value'].values
carbs = df['carbohydrates-g_value'].values
sugars = df['sugars_g_value'].values
protien = df['proteins-g_value'].values
salt = df['salt-g_value'].values
sodium = df['sodium-g_value'].values


In [89]:

N = len(weight_food)
model = ro.Model('Load Optimization')

x = model.dvar((N), "I")  # x[i]: the number of product I
w = weight_food.values

# objectives
model.max(w @ x)

In [90]:
# constrains
model.st(sum(x[i] * calories[i] for i in range(N))<= total_cal*10)
model.st(sum(x[i] * fat[i] for i in range(N)) <= total_fat*10)
model.st(sum(x[i] * sat_fat[i] for i in range(N)) <= total_satfat*10)
model.st(sum(x[i] * carbs[i] for i in range(N)) <= total_carbs*10)
model.st(sum(x[i] * sugars[i] for i in range(N)) <= total_sugars*10)
model.st(sum(x[i] * protien[i] for i in range(N)) <= total_proteins*10)
model.st(sum(x[i] * salt[i] for i in range(N)) <= total_salt*10)
model.st(sum(x[i] * sodium[i] for i in range(N)) <= total_sodium*10)

model.st(sum(x[i] * w[i] for i in range(N)) <= total_weight * 1000)
model.st( x[i] >= 0 for i in range(N) )

model.solve(grb)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Being solved by Gurobi...
Solution status: 2
Running time: 0.0153s


In [91]:
print("Total weight of food products: {}".format(model.get()/1000))
for i in range(N):
    if x.get()[i] > 0:
        pdtName = df.loc[df['pdt_Id'] == i, 'product_name'].item()
        print( "Product ID: {:10} \
            Product Name: {:30} \
            Value: {:30}".format(i, pdtName, x.get()[i]))

Total weight of food products: 500.0
Product ID:          0             Product Name: Straight Red Tea                           Value:                         2000.0
